<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/binary_classification_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델의 depth와 활성함수의 중첩, 그에 따른 비선형성 증가의 의미.

딥러닝 모델의 깊어질수록 활성함수가 중첩되고 모델전체의 비선형성이 증가하게 된다. 비선형성이 증가함에따라 모델이 그리는 곡면은 더 복잡해지게 되므로 SGD같이 local minimum을 탈출할 수단이 없는 optimizer는 local minimum에 빠질 확률이 커지게 되고 sigmoid같이 모든지점에서 기울기가 1이하인 activation function을 갖는 모델은 vanishing gradient문제가 생길 것 이다. 이번 프로젝트를 통해 모델의 깊이에따른 학습정체를 확인하고 그 원인이 vanishing gradient때문인지 아니면 local minimum때문인지 분석해본다.

**가정1 vanishing gradient가 원인인 경우**
\
\
실험1

활성화 함수로 Sigmoid 함수를 사용할 경우 vanishing gradient로 학습이 제대로 이뤄지지 않을것이다. 반면 gradient를 잘 보존하는 ReLU를 사용할 경우 학습이 잘 이뤄이질 것 이다.
\
\
\
**가정2 local minimum이 원인인 경우**
\
\
실험1

학습정체의 원인이 local minimum인 경우 모델이 완전히 local minimum에 묶였다고 생각될때 gradient값을 확인해 보면 거의 0에 수렴할 것 이다.
\
\
조건2 활성함수로 ReLU를 사용할 경우

gradient가 잘 보존되어도 sgd특성상 local minimum을 빠져나올 수 없음.
\
\
조건3

local minimum으로 완전히 수렴한후엔 learning rate를 매우크게 줘도 local munimumdptj 빠져나올 수 없음.
\
\
조건4

momentum등 local minimum을 탈출할 수 있는 optimizer를 사용하면 최적화 가능.

# Data 생성

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# case 1 키, 몸무게
N=20
random0=torch.randn(int(N/2),1)
random5=torch.randn(int(N/2),1)+8

class1_data=torch.hstack([random0,random5])
class2_data=torch.hstack([random5,random0])
# class3_data=torch.hstack([random5,random0+8])

class1_label=torch.ones(int(N/2),1)
class2_label=torch.zeros(int(N/2),1)
class3_label=torch.ones(int(N/2),1)

X=torch.vstack([class1_data,class2_data])
y=torch.vstack([class1_label,class2_label])

# X=torch.vstack([class1_data,class2_data,class3_data])
# y=torch.vstack([class1_label,class2_label,class3_label])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(class1_data[:,0],class1_data[:,1],'bo')
plt.plot(class2_data[:,0],class2_data[:,1],'ro')
# plt.plot(class3_data[:,0],class3_data[:,1],'bo')
plt.xlabel('x1')
plt.ylabel('x2')
plt.grid()

# 가정1 - 실험1

Sigmoid와 ReLU비교를 위한 모델 생성

In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Sequential(nn.Linear(2, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 1), nn.Sigmoid())

        self.linear2 = nn.Sequential(nn.Linear(2, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 10), nn.ReLU(),
                                    nn.Linear(10, 1), nn.Sigmoid())

    def forward(self, x):

        x1 = self.linear1(x)
        x2 = self.linear2(x)

        return x1, x2

gradient 관찰.

ReLU가 Sigmoid보다 gradient를 잘 보존하지만 ReLU만 사용해서는 vanishing gradient현상을 완전히 극복할수는 없는 것 같다.
\
\
! 파라미터가 너무적은(나의 경우 5x5 Linear 레이어)레이어를 깊게 쌓다보면 가끔 ReLU를 사용한 모델의 gradient가 어느순간 0으로 수렴 후 회복하지 못 하는 현상이 생긴다. 음수값에서 gradient를 0으로 반환하는 ReLU함수의 특성으로 생기는 현상인 듯 하다. 학습과정에서 이러한 현상이 생기면 안되므로 각 레이어의 파라미터 수를 10x10으로 수정했다.

In [ ]:
model1 = Model1()
model1.train()
optimizer = optim.SGD(model1.parameters(), lr = 1e-1)
y_hat1, y_hat2 = model1(X)
loss1 = F.binary_cross_entropy(y_hat1, y)
loss2 = F.binary_cross_entropy(y_hat2, y)
optimizer.zero_grad()
loss1.backward()
loss2.backward()

In [ ]:
weight_norm1 = []
weight_norm2 = []

for i in range(0, 35, 2):
    w_norm1 = torch.sum(torch.abs(model1.linear1[i].weight.grad))
    w_norm2 = torch.sum(torch.abs(model1.linear2[i].weight.grad))

    weight_norm1 += [w_norm1]
    weight_norm2 += [w_norm2]

In [ ]:
plt.plot(range(0,10,2), weight_norm1[0:5], 'ro', label = 'Sigmoid')
plt.plot(range(0,10,2), weight_norm2[0:5], 'bo', label = 'LeLU')

plt.xlabel('Layer_Index')
plt.ylabel('Weighy_Matrix_Norm')

plt.legend()

plt.show()

print(weight_norm1)
print(weight_norm2)

학습과정 관찰.

Sigmoid를 활성화함수를 사용하면 vanishing gradient 현상이 발생하지만 ReLU를 사용하면 극복할 수 있는 적당한 깊이의 모델을 사용해 가정1을 실험해본다.
\
\
! 초기에 아주깊은 신경망을 생성한후 실험했을땐 Sigmoid와 ReLU모두 Loss값 0.6 부근에서 학습이 정체되어 가정1이 틀렸다고 생각했지만 ReLU가 vanishing gradient를 완전히 극복할 수 없음을 알고 적당한 깊이로 실험한 결과 Sigmoid를 사용한 모델은 학습이 정체되는 반면 ReLU를 사용하면 학습이 잘이루어지는것을 관찰했다.

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Sequential(nn.Linear(2, 10), nn.Sigmoid(),
                                     nn.Linear(10, 10), nn.Sigmoid(),
                                     nn.Linear(10, 10), nn.Sigmoid(),
                                     nn.Linear(10, 10), nn.Sigmoid(),
                                     nn.Linear(10, 10), nn.Sigmoid(),
                                     nn.Linear(10, 10), nn.Sigmoid(),
                                     nn.Linear(10, 1), nn.Sigmoid())

        self.linear2 = nn.Sequential(nn.Linear(2, 10), nn.ReLU(),
                                     nn.Linear(10, 10), nn.ReLU(),
                                     nn.Linear(10, 10), nn.ReLU(),
                                     nn.Linear(10, 10), nn.ReLU(),
                                     nn.Linear(10, 10), nn.ReLU(),
                                     nn.Linear(10, 10), nn.ReLU(),
                                     nn.Linear(10, 1), nn.Sigmoid())

    def forward(self, x):

        x1 = self.linear1(x)
        x2 = self.linear2(x)

        return x1, x2

In [ ]:
LR = 1e-1
EPOCH = 1000

model2 = Model2()

optimizer = optim.SGD(model2.parameters(), lr=LR)
# optimizer = optim.Adam(model.parameters(), lr=LR)

loss_history1 =[]
loss_history2 =[]

model2.train()
for ep in range(EPOCH):

    y_hat1, y_hat2 = model2(X)

    loss1 = F.binary_cross_entropy(y_hat1, y)
    loss2 = F.binary_cross_entropy(y_hat2, y)

    optimizer.zero_grad()
    loss1.backward()
    loss2.backward()
    optimizer.step()

    loss_history1 += [loss1.item()]
    loss_history2 += [loss2.item()]

plt.plot(range(1,EPOCH+1),loss_history1, label = 'Sigmoid')
plt.plot(range(1,EPOCH+1),loss_history2, label = 'ReLU')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend()

plt.show()

# 가정2-실험2

학습정체를 일으키고 gradient를 확인하기 위한 모델을 생성

In [ ]:
class Model3(nn.Module()):

    def __init__(self):
        super().__init__()

        self.linear = nn.sequential(nn.Linear(2, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 10), nn.Sigmoid(),
                                    nn.Linear(10, 1), nn.Sigmoid())

    def forward(self, x):

        x = self.linear(x)

        return x

In [ ]:
LR = 1e-1
EPOCH = 1000

model3 = Model3()

optimizer = optim.SGD(model3.parameters(), lr=LR)
# optimizer = optim.Adam(model.parameters(), lr=LR)

loss_history3 =[]

model2.train()
for ep in range(EPOCH):

    y_hat3 = model3(X)

    loss3 = F.binary_cross_entropy(y_hat3, y)

    optimizer.zero_grad()
    loss3.backward()
    optimizer.step()

    loss_history3 += [loss3.item()]

plt.plot(range(1,EPOCH+1),loss_history3, label = 'Sigmoid')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend()

plt.show()

In [ ]:
# x1_test=torch.linspace(-10,10,30) # case 1
# x2_test=torch.linspace(-10,10,30) # case 1
x1_test = torch.linspace(-10,10,30)
x2_test = torch.linspace(-10,10,30)

X1_test, X2_test = torch.meshgrid(x1_test, x2_test)
# print(X1_test.shape)

X_test = torch.cat([X1_test.unsqueeze(dim=2), X2_test.unsqueeze(dim=2)], dim=2)
# print(X_test)

model.eval()

with torch.no_grad():
    y_hat = model(X_test)

Y_hat = y_hat.squeeze()


plt.figure(figsize=[10, 9]) # figsize=[가로, 세로]
ax = plt.axes(projection="3d")
ax.view_init(elev=25,azim=-140)
ax.plot_surface(X1_test,X2_test, Y_hat.numpy(), cmap="viridis", alpha=0.2)
plt.plot(class1_data[:,0],class1_data[:,1],class1_label.squeeze(),'bo')
plt.plot(class2_data[:,0],class2_data[:,1],class2_label.squeeze(),'ro')
# plt.plot(class3_data[:,0],class3_data[:,1],class3_label.squeeze(),'bo')
plt.xlabel("x1")
plt.ylabel("x2")